In [4]:
from pathlib import Path
import numpy as np
import xarray as xr
import functools
import multiprocessing as mp
import os


project_root = Path('/home/glen/works/forcing_generate/')
merged_netcdf = 'RDRSv2.1_UTC0.nc'


rdrs = xr.open_dataset(project_root / merged_netcdf)

one_dim = len(rdrs['rlat']) * len(rdrs['rlon'])
variables = list(rdrs.keys())
variables.remove('rotated_pole')

In [5]:
# Save each variable to a numpy binary file (only used once)
@functools.lru_cache(maxsize = None)
def save_to_array(var : str):
    huge_ncds = xr.open_dataset(project_root / merged_netcdf)
    npy_path = project_root
    if not os.path.isdir(npy_path):
        os.makedirs(npy_path)
    flat = huge_ncds[var].to_numpy().reshape(len(huge_ncds['time']), one_dim)
    np.save(npy_path / f'{var}.npy', flat)


def save_to_array_clear():
    save_to_array.cache_clear()


def save_to_array_wrapper(var : str):
    save_to_array(var)
    save_to_array_clear()


pool = mp.Pool(processes = 4)
pool.map(save_to_array_wrapper, variables)